In [1]:
import subprocess

def model(state, action):
    result = subprocess.run(
        ["/home/griffin/driverless/driverless_ws/build/controls/model_wrapper"] + list(map(str, (state + action))),
        capture_output=True)
    return list(map(float, result.stdout.decode("UTF-8").split(" ")[:10]))

in_dim_names = [
    "x",
    "y",
    "yaw",
    "x_car_dot",
    "y_car_dot",
    "yaw_dot",
    "x_moment",
    "downforce",
    "omega_f",
    "omega_r",
    "swangle",
    "torque_f",
    "torque_r"
]

out_dim_names = list(map(lambda x: x + "_dot", in_dim_names))

In [2]:
print(model([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0]))

[1.0, 0.0, 0.0, -11.3482, -0.956411, -3.49554, 0.0, 0.0, 691.881, 791.794]


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from copy import deepcopy

%matplotlib qt

def plot_surface(indep_idxs, dep_idx, indep_min, indep_max, base):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')


    def fun(indep1, indep2):
        global model
        
        to_pass = deepcopy(base)
        to_pass[indep_idxs[0]] = indep1
        to_pass[indep_idxs[1]] = indep2

        return model(to_pass[:10], to_pass[-3:])[dep_idx]
    
    x = np.linspace(indep_min[0], indep_max[0], 10)
    y = np.linspace(indep_min[1], indep_max[1], 10)
    X, Y = np.meshgrid(x, y)
    xr, yr = np.ravel(X), np.ravel(Y)
    zs = np.array([fun(x,y) for x, y in zip(xr, yr)])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z)

    ax.set_xlabel(in_dim_names[indep_idxs[0]])
    ax.set_ylabel(in_dim_names[indep_idxs[1]])
    ax.set_zlabel(out_dim_names[dep_idx])

In [7]:
plot_surface([10, 9], 3, [-0.5, 0.5], [0.5, 1.5], [0, 0, 0, 0.2286, 0, 0, 0, 0, 1, 1, 0, 0, 0])
plt.show()

In [9]:
plot_surface([10, 9], 3, [-0.5, 0.5], [0.5, 1.5], [0, 0, 0, 0.2286, 0.1, 0, 0, 0, 1, 1, 0, 0, 0])
plt.show()